In [2]:
#Part 1: The Basic Tools of Private, Decentralized Data Science

import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5]) #create and print a tensor

tensor([1, 2, 3, 4, 5])

In [3]:
#Section 1.1 - Sending Tensors to Bob's Machine
x = torch.tensor([1,2,3,4,5]) #create tensor x
y = x + x #add tensor x to itself
print(y)

tensor([ 2,  4,  6,  8, 10])


In [4]:
bob = sy.VirtualWorker(hook, id="bob") #create virtual worker bob

x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

x_ptr = x.send(bob)
y_ptr = y.send(bob)

x_ptr

(Wrapper)>[PointerTensor | me:30118607386 -> bob:21119658353]

In [5]:
bob._objects

{21119658353: tensor([1, 2, 3, 4, 5]), 36411192608: tensor([1, 1, 1, 1, 1])}

In [6]:
z = x_ptr + x_ptr
z

(Wrapper)>[PointerTensor | me:75994159071 -> bob:80790646071]

In [7]:
bob._objects

{21119658353: tensor([1, 2, 3, 4, 5]),
 36411192608: tensor([1, 1, 1, 1, 1]),
 80790646071: tensor([ 2,  4,  6,  8, 10])}

In [8]:
x_ptr

(Wrapper)>[PointerTensor | me:30118607386 -> bob:21119658353]

In [9]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [10]:
bob

<VirtualWorker id:bob #objects:3>

In [11]:
bob == x_ptr.location

True

In [12]:
x_ptr.id_at_location

21119658353

In [13]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

In [14]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [15]:
me == x_ptr.owner

True

In [16]:
x_ptr

(Wrapper)>[PointerTensor | me:30118607386 -> bob:21119658353]

In [17]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [18]:
y_ptr

(Wrapper)>[PointerTensor | me:88374305155 -> bob:36411192608]

In [19]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [20]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [21]:
bob._objects

{}

In [22]:
#Section 1.2 - Using Tensor Pointers

#Creates tensors x and y and sends them to bob returning the pointer to the tensor
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

#Serializes the command and sends it to be executed on bob returning a pointer to the resulting tensor z
z = x + y

z

(Wrapper)>[PointerTensor | me:32054131929 -> bob:29458013200]

In [23]:
z.get()

tensor([2, 3, 4, 5, 6])

In [24]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:78056161971 -> bob:54922909875]

In [25]:
z.get()

tensor([2, 3, 4, 5, 6])

In [26]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

z = (x + y).sum()
z.backward()

(Wrapper)>[PointerTensor | me:49385875586 -> bob:5327304487]

In [27]:
x = x.get()
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [28]:
x.grad

tensor([1., 1., 1., 1., 1.])